# Projekt ZTNBD

## Lokalna instalacja
###### Wymagania
- docker
- ~4GB wolnego miejsca na dysku
- zalecany linux + build-essential

###### Linux 
```bash
make notebook
```
###### Others (nie sprawdzane)
```bash
# lokalizacja - główny katalogu projektu
docker run -it --rm -v $(pwd):/home/jovyan/work -p 8888:8888 jupyter/pyspark-notebook
```

Komenda startuje kontener dockerowy z Jupyterem i podmontowuje katalog projektu.
Moduły znajdują się w katalogu `modules` i tam też będą lądowały kolejne.
Uruchomienie póki co możliwe jest tylko z poziomu notebook'a.

## Moduły

##### PostTransformer
Klasa PostTransformer dziedziczy po klasach pyspark.ml.Transformer, pyspark.ml.param.shared.HasInputCol, pyspark.ml.param.shared.HasOutputCol. Posiada metodę transform, która przyjmuje na wejściu obiekt typu dataframe. Metoda ta wydobywa z kolumny inputCol, z formatu json, content i umieszcza go w kolumnie outputCol w postaci tekstu.
##### TranslateTransformer
Klasa TranslateTransformer dziedziczy po klasach pyspark.ml.Transformer, pyspark.ml.param.shared.HasInputCol, pyspark.ml.param.shared.HasOutputCol. Posiada metodę transform, która przyjmuje na wejściu obiekt typu dataframe. Metoda ta tłumaczy tekst zawarty w kolumnie inputCol  z języka polskiego na angielski i umieszcza go w kolumnie outputCol.
##### SentenceTransformer
Klasa SentenceTransformer dziedziczy po klasach pyspark.ml.Transformer, pyspark.ml.param.shared.HasInputCol, pyspark.ml.param.shared.HasOutputCol. Posiada metodę transform, która przyjmuje na wejściu obiekt typu dataframe. Metoda ta dzieli tekst zawarty w kolumnie inputCol  na zdania i umieszcza go w kolumnie outputCol w postaci tablicy tekstów.
##### SpeechPartsTransformer
Klasa SpeechPartsTransformer dziedziczy  po klasach pyspark.ml.Transformer, pyspark.ml.param.shared.HasInputCol, pyspark.ml.param.shared.HasOutputCol. Posiada metodę transform, która przyjmuje na wejściu obiekt typu dataframe. Metoda ta z tekstu zawartego w kolumnie inputCol  zlicza wystąpienie części mowy i wstawia do outputCol w postaci jsona.
##### SentimentTransformer
Klasa SentimentTransformer dziedziczy  po klasach pyspark.ml.Transformer, pyspark.ml.param.shared.HasInputCol, pyspark.ml.param.shared.HasOutputCol. Posiada metodę transform, która przyjmuje na wejściu obiekt typu dataframe. Metoda ta z tekstu zawartego w kolumnie inputCol  wylicza sentiment i wstawia do kolumny outputCol.

## Dokumentacja
[Google Docs](https://docs.google.com/document/d/1IylTvJbRe8s_j_bZqbM-6nWVa2IQDjQiZx-NyJsGZbg)

 


In [1]:
!pip install textblob
!python -m textblob.download_corpora lite

[nltk_data] Downloading package brown to /home/jovyan/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Finished.


In [2]:
import json

import pyspark
import pyspark.sql
from pyspark.ml import Pipeline

from modules.posts import (
    SentenceTransformer, PostTransformer, TranslateTransformer,
    SpeechPartsTransformer, SentimentTransformer
)
from modules.features import (
    FeaturesTransformer
)


sc = pyspark.SparkContext('local[*]', 'PipelineFlow')
sess = pyspark.sql.SparkSession(sc)
    

In [3]:

def transform_features(spark_context):
    rdd = spark_context.wholeTextFiles('data/featuresample.json')
    rdd = rdd.map(lambda x: (x[0], x[1]))
    df = rdd.toDF(['file', 'content'])

    features = [
        "leaf",
        "has-attribute-class",
    ]
    
    feature_transformer = FeaturesTransformer(features=features)
    feature_transformer.setInputCol('content').setOutputCol('features')
    
    stages = [
        feature_transformer,
    ]
    
    pipeline = Pipeline(stages=stages)
    result = pipeline.fit(df).transform(df)
    return result


In [ ]:

def transform_posts(spark_context):
    rdd = spark_context.wholeTextFiles('data/posts/*')
    rdd = rdd.map(lambda x: (x[0], json.loads(x[1])))
    df = rdd.toDF(['file', 'content'])
    
    poster = PostTransformer()
    poster.setInputCol('content').setOutputCol('posts')
    
    translator = TranslateTransformer()
    translator.setInputCol('posts').setOutputCol('translated')
    
    sentencer = SentenceTransformer()
    sentencer.setInputCol('translated').setOutputCol('sentences')
    
    speech_parter = SpeechPartsTransformer()
    speech_parter.setInputCol('translated').setOutputCol('speechParts')
    
    sentimenter = SentimentTransformer()
    sentimenter.setInputCol('translated').setOutputCol('sentiments')

    stages = [
        poster,
        translator, 
        sentencer, 
        speech_parter, 
        sentimenter
    ]
    
    pipeline = Pipeline(stages=stages)
    result = pipeline.fit(df).transform(df)
    return result


In [ ]:
out = transform_posts(sc)

a = out.select('sentences').first().sentences[0]
b = out.select('sentences').first().sentences[1]
c = out.select('sentences').first().sentences[2]
d = out.select('translated').first().translated[0]
e = out.select('speechParts').first().speechParts
f = out.select('sentiments').first().sentiments[0]
g = out.select('sentiments').first().sentiments[1]
h = out.select('sentiments').first().sentiments[2]

scheme = '{}\n\n{}\n\n{}\n\n{}\n\n{}\n\n{}\n\n{}\n\n{}'
print(scheme.format(a,b,c,d,e,f,g,h))

In [ ]:
result = transform_features(sc)

print(result.first().features)